In [1]:
! pip install pandas \
    matplotlib

In [2]:
import pandas as pd
import glob as gb

In [ ]:
DIR = 'airflow/logs/dag_id=app_template/*/*/*'
logs_paths = gb.glob(DIR)

df_logs = pd.DataFrame()
for path in logs_paths:
    df = pd.read_csv(path, sep=' ', skiprows=6, header=None)
    df = df[df[1]=='{helpers.py:110}']
    col_names = {
        0: 'timestamp',
        4: 'tasks',
        5: 'batch_id',
        6: 'processed_%',
        7: 'seconds'
    }
    df = df[col_names.keys()]
    df.rename(columns=col_names, inplace=True)
    task_type = path.split('/')[4].split('=')[1].split('.')[0]
    df['task_type'] = task_type
    run_id = path.split('/')[3].split('=')[1].split('__')[1]
    df['run_id'] = run_id
    df_logs = pd.concat([df_logs, df], ignore_index=True)
df_logs.reset_index(drop=True, inplace=True)

df_logs['seconds'] = pd.to_numeric(df_logs['seconds'], errors='coerce')
df_logs['processed_%'] = pd.to_numeric(df_logs['processed_%'], errors='coerce')
df_logs['timestamp'] = \
    pd.to_datetime(df_logs['timestamp'].str.strip('[]'), errors='coerce')

df_logs.to_csv('tmp/logs.csv', index=False)

display(df_logs.shape)
df_logs.sample(10)

(3692, 7)

,timestamp,tasks,batch_id,processed_%,seconds,task_type,run_id
2678,2025-06-08 16:12:21.837000+00:00,1-1-3,22,53.66,2.029,loaders,dc3dd13e-933a-4fd1-918b-75e13ff1cb1b
403,2025-06-08 16:27:25.267000+00:00,1-1-5,5,20.00,2.270,loaders,c2e453ee-5c19-4a7f-8dd7-2020810bd5f8
98,2025-06-08 16:19:44.792000+00:00,1-1-4,6,4.80,3.630,transformers,9290d4f5-5dad-4e54-b870-69ef3bff7d12
1478,2025-06-08 15:59:25.368000+00:00,1-1-1,107,85.60,2.300,extractors,46e7fd0e-0f28-4e4f-813c-3e444d476c7b
2104,2025-06-08 16:49:32.729000+00:00,1-2-4,17,54.84,2.602,loaders,97441767-9ed6-40e3-80be-679c20758fec
2015,2025-06-08 16:52:13.389000+00:00,1-2-4,52,83.87,3.589,transformers,97441767-9ed6-40e3-80be-679c20758fec
2889,2025-06-08 16:12:20.278000+00:00,1-1-3,26,20.80,2.136,extractors,dc3dd13e-933a-4fd1-918b-75e13ff1cb1b
320,2025-06-08 16:21:57.986000+00:00,1-1-4,72,57.60,2.487,extractors,9290d4f5-5dad-4e54-b870-69ef3bff7d12
2435,2025-06-08 16:47:19.328000+00:00,1-2-3,48,77.42,3.687,transformers,6729361f-0b1c-499a-97b7-c418ed90e7c8
2704,2025-06-08 16:11:53.347000+00:00,1-1-3,7,5.60,3.158,transformers,dc3dd13e-933a-4fd1-918b-75e13ff1cb1b


In [ ]:
cols = ['extractors', 'transformers', 'loaders']
df_time_span = (df_logs
                .groupby(['run_id', 'task_type'])['timestamp']
                .agg(start_time='min', end_time='max')
                .reset_index())
df_time_span['duration'] = \
    (df_time_span['end_time'] - df_time_span['start_time']).dt.total_seconds()
df_grouped = \
    (df_time_span
     .pivot(index='run_id', columns='task_type', values='duration')
     .fillna(0)
     .reset_index())
df_grouped['total_pipeline_time'] = df_grouped[cols].max(axis=1)
config_map = (df_logs[['run_id', 'tasks']]
              .drop_duplicates()
              .rename(columns={'tasks': 'config'}))
df_summary = df_grouped.merge(config_map, on='run_id').sort_values(by='config')
df_summary[['run_id']+cols+['total_pipeline_time','config']]

,run_id,extractors,transformers,loaders,total_pipeline_time,config
0,46e7fd0e-0f28-4e4f-813c-3e444d476c7b,272.700,467.352,263.234,467.352,1-1-1
6,a70bf2db-3afb-4197-b0c9-72bf6d2a7c27,267.874,462.175,153.097,462.175,1-1-2
9,dc3dd13e-933a-4fd1-918b-75e13ff1cb1b,257.064,463.726,93.363,463.726,1-1-3
4,9290d4f5-5dad-4e54-b870-69ef3bff7d12,265.908,460.567,88.989,460.567,1-1-4
8,c2e453ee-5c19-4a7f-8dd7-2020810bd5f8,263.331,464.022,59.778,464.022,1-1-5
2,72d90dd1-eaab-4415-990e-40269e13c5c6,264.287,250.912,266.522,266.522,1-2-1
7,adae183b-f26b-47af-b674-dc3354b18ae1,265.814,231.141,146.577,265.814,1-2-2
1,6729361f-0b1c-499a-97b7-c418ed90e7c8,264.512,228.892,102.216,264.512,1-2-3
5,97441767-9ed6-40e3-80be-679c20758fec,266.669,226.844,85.154,266.669,1-2-4
3,8a1afd80-e110-49bb-9a55-a031af4bbd88,203.336,199.704,67.236,203.336,1-2-5
